In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

frames = cv2.imread('./test4.jpg')
stretch = frames.copy() #Frame is the imported image
#Increasing the contrast of the image
# CLAHE (Contrast Limited Adaptive Histogram Equalization)
clahe = cv2.createCLAHE(clipLimit=2., tileGridSize=(8,8))
lab = cv2.cvtColor(stretch, cv2.COLOR_BGR2LAB)  # convert from BGR to LAB color space
l, a, b = cv2.split(lab)  # split on 3 different channels

l2 = clahe.apply(l)  # apply CLAHE to the L-channel

lab = cv2.merge((l2,a,b))  # merge channels
img2 = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)  # convert from LAB to BG


# convert to gray
gray = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
# blur
smooth = cv2.GaussianBlur(gray, (11,11), 0)

img2 = cv2.cvtColor(smooth,cv2.COLOR_GRAY2BGR)

frame_HSV = cv2.cvtColor(img2, cv2.COLOR_BGR2HSV)
frame_threshold = cv2.inRange(frame_HSV, (0, 0, 10), (180, 255, 80))
contours,hierarchy, = cv2.findContours(frame_threshold,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

# areas = [cv2.contourArea(c) for c in contours]
# sorted_areas = np.sort(areas)
# #cnt=contours[areas.index(sorted_areas[-1])] #the biggest contour
# largestContourArea = sorted_areas[-1]*0.01
# largestContour = 0
# for cnt in contours:
#     contourArea = cv2.contourArea(cnt)
#     if( contourArea > largestContourArea):
#         #cv2.drawContours(frames, cnt, -1, (255,0,132), thickness=4) 
#         x,y,w,h = cv2.boundingRect(cnt)
#         cv2.rectangle(frames, (x, y), (x+w, y+h), (255,0,0), 4)
        
#         ROI = frames[y:y+h,x:x+w]     
#         plt.imshow(ROI)
#         plt.show()
# cv2.drawContours(frames, contours, -1, (0,150,132), thickness=4) 
# plt.imshow(frames)
# plt.show()

In [ ]:
import cv2
import numpy as np

def expand_contour(contour, image, similarity_threshold=20, dilation_size=5):
    # Create a mask for the contour
    contour_mask = np.zeros_like(image, dtype=np.uint8)
    cv2.drawContours(contour_mask, [contour], -1, (255, 255, 255), cv2.FILLED)

    # Convert the image to Lab color space
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # Get the average color within the contour
    avg_color = np.mean(lab_image[np.where(contour_mask[:, :, 0] == 255)], axis=0)

    # Calculate the color difference using Euclidean distance
    color_diff = np.sqrt(np.sum((lab_image - avg_color) ** 2, axis=2))

    # Create a mask for similar color pixels
    similarity_mask = color_diff <= similarity_threshold

    # Dilate the similarity mask
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (dilation_size, dilation_size))
    dilated_mask = cv2.dilate(similarity_mask.astype(np.uint8), kernel, iterations=1)

    # Expand the contour by combining the contour mask and dilated mask
    expanded_mask = np.logical_or(contour_mask[:, :, 0], dilated_mask)

    # Find the new contour from the expanded mask
    contours, _ = cv2.findContours(expanded_mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) > 0:
        return contours[0]  # Return the first (largest) expanded contour
    else:
        return None


# Example usage:
# Assuming 'image' is the original image and 'contour' is the contour you want to expand

# Specify the similarity threshold and dilation size (adjust as needed)
# similarity_threshold = 20
# dilation_size = 5

# Expand the contour
# expanded_contour = expand_contour(contour, image, similarity_threshold, dilation_size)

# Draw the expanded contour on the image
# cv2.drawContours(image, [expanded_contour], -1, (0, 255, 0), 2)

# # Display the image with the expanded contour
# cv2.imshow('Expanded Contour', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [ ]:
for contour in contours:
    expanded_contour = expand_contour(contour, frames, 20, 5)

    # Draw the expanded contour on the image
    cv2.drawContours(frames, [expanded_contour], -1, (0, 255, 0), 2)

# Display the image with the expanded contour
cv2.imshow('Expanded Contour', frames)
cv2.waitKey(0)
cv2.destroyAllWindows()